# **THE KNIGHT’S TOUR PROBLEM**


## Penjelasan Masalah
  Knight's Tour Problem adalah masalah klasik dalam teori graf yang berkaitan dengan Hamiltonian Path. Masalah ini menanyakan: apakah sebuah bidak kuda dapat mengunjungi semua kotak pada papan catur (8×8) tepat satu kali dengan menggunakan pergerakan kuda (bentuk L)?

## Warnsdorff's Heuristic Algorithm
### Warnsdorff's Rule:

  "Selalu pindah ke kotak yang belum dikunjungi dengan minimal degree (jumlah kemungkinan langkah berikutnya paling sedikit)"

### Intuisi:
  Kotak dengan sedikit pilihan lebih berisiko menjadi jalan buntu, jadi kunjungi lebih awal.

### Time Complexity: O(n²)
- Setiap langkah memiliki maksimal 8 pilihan
- Untuk setiap pilihan, hitung degree: O(8)
- Total: n² × 8 × 8 = O(n²)

### Space Complexity: O(n²)
- Untuk menyimpan papan

## Implementasi Code:

In [16]:
def get_knight_moves():
    return [
        (2, 1),  (1, 2),  (-1, 2), (-2, 1),
        (-2, -1), (-1, -2), (1, -2), (2, -1)
    ]


Fungsi di atas hanya mengembalikan 8 kemungkinan vektor gerak kuda.

Tiap pasangan (dx, dy) artinya:
- dx: perpindahan baris
- dy: perpindahan kolom

Contoh: (2, 1) berarti pindah 2 kotak ke bawah dan 1 kotak ke kanan

In [17]:
def is_safe(x, y, board):
    n = len(board)
    return 0 <= x < n and 0 <= y < n and board[x][y] == -1


Mengecek apakah koordinat (x, y):
- Masih di dalam batas papan (0 … n-1), dan
- Belum pernah dikunjungi (board[x][y] == -1).

Digunakan untuk menghindari:
- Index out of range
- Mengunjungi kotak yang sama lebih dari sekali.

In [18]:
def count_onward_moves(x, y, board, moves):
    count = 0
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if is_safe(nx, ny, board):
            count += 1
    return count


Fungsi tersebut menghitung berapa banyak langkah valid yang bisa dilakukan dari sebuah kotak (x, y).

Nilai ini disebut degree:
- Semakin kecil degree, semakin “sempit” pilihan jalannya.
- Heuristik Warnsdorff: pilih kotak dengan degree paling kecil dulu supaya tidak buntu di akhir.

In [19]:
def get_next_moves_sorted(x, y, board, moves):
    candidates = []

    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if is_safe(nx, ny, board):
            degree = count_onward_moves(nx, ny, board, moves)
            candidates.append((degree, nx, ny))

    candidates.sort(key=lambda item: item[0])
    return [(nx, ny) for degree, nx, ny in candidates]


Mencari semua langkah berikutnya yang valid dari (x, y).

Untuk setiap langkah (nx, ny):
- Hitung degree dengan count_onward_moves.
- Simpan sebagai tuple (degree, nx, ny).

Urutkan list candidates berdasarkan degree (ascending).

Return hanya koordinat (nx, ny) yang sudah terurut.

Merupakan implementasi dari Warnsdorff’s Rule.

In [28]:
def knights_tour_warnsdorff(n=8, start_x=0, start_y=0):
    board = [[-1 for _ in range(n)] for _ in range(n)]

    moves = get_knight_moves()

    x, y = start_x, start_y
    board[x][y] = 0  # langkah pertama

    for step in range(1, n * n):
        next_positions = get_next_moves_sorted(x, y, board, moves)

        if not next_positions:
            return None  # dead-end, tour gagal

        x, y = next_positions[0]  # pilih kotak dengan degree terkecil
        board[x][y] = step

    return board


Input:
- n: ukuran papan (default 8 untuk 8×8).
- start_x, start_y: posisi awal kuda (indeks 0-based).

board:
- -1 artinya kotak belum dikunjungi.
- Angka 0..n*n-1 menyatakan urutan kunjungan.

Langkah-langkah:
1. Set posisi awal dengan 0.
2. Ulangi dari step = 1 sampai step = n*n - 1:
  - Ambil semua langkah valid dengan get_next_moves_sorted.
  - Jika kosong → tidak ada jalan lagi → return None.
  - Ambil posisi pertama (degree terkecil) sebagai (x, y) baru.
  - Isi board[x][y] = step.
3. Jika loop selesai tanpa macet → semua kotak dikunjungi → return board.

In [25]:
def print_board(board):
    if board is None:
        print("Tidak ada solusi ditemukan.")
        return

    n = len(board)
    max_val = n * n - 1
    width = len(str(max_val))

    print("\nUrutan perjalanan kuda:")
    print("=" * (n * (width + 2)))

    for row in board:
        print(" ".join(f"{val:{width}d}" for val in row))

    print("=" * (n * (width + 2)))


Hanya untuk memformat output supaya rapi dan Membantu melihat jalur dengan mudah.


In [26]:
def visualize_path_simple(board):
    if board is None:
        return

    n = len(board)
    total = n * n
    print("\nVisualisasi jalur (S = start, E = end):")

    for i in range(n):
        row_str = ""
        for j in range(n):
            if board[i][j] == 0:
                row_str += " S"
            elif board[i][j] == total - 1:
                row_str += " E"
            else:
                row_str += " ·"
        print(row_str)


Menandai kotak start dengan S.

Menandai kotak end dengan E.

Kotak lainnya sebagai ·.

Cukup untuk menunjukkan bahwa seluruh papan terisi dan kita tahu posisi awal/akhir.

In [31]:
if __name__ == "__main__":
    n = 8
    start_x, start_y = 0, 0

    print(f"Mencari Knight's Tour {n}x{n} dari posisi awal ({start_x}, {start_y}) "
          "dengan Warnsdorff's Heuristic...\n")

    tour = knights_tour_warnsdorff(n=n, start_x=start_x, start_y=start_y)

    if tour:
        print("Solusi ditemukan!\n")
        print_board(tour)
        visualize_path_simple(tour)
    else:
        print("Tour gagal ditemukan. Coba posisi awal lain.")


Mencari Knight's Tour 8x8 dari posisi awal (0, 0) dengan Warnsdorff's Heuristic...

Solusi ditemukan!


Urutan perjalanan kuda:
 0 33  2 17 48 31 12 15
 3 18 55 32 13 16 49 30
56  1 34 47 54 51 14 11
19  4 59 52 35 46 29 50
40 57 36 45 60 53 10 25
 5 20 41 58 37 26 63 28
42 39 22  7 44 61 24  9
21  6 43 38 23  8 27 62

Visualisasi jalur (S = start, E = end):
 S · · · · · · ·
 · · · · · · · ·
 · · · · · · · ·
 · · · · · · · ·
 · · · · · · · ·
 · · · · · · E ·
 · · · · · · · ·
 · · · · · · · ·
